# CityBikes

Send a request to CityBikes for the city of your choice. 

In [142]:
import pandas as pd
import numpy as np
import requests
import os

from pprint import pprint

In [143]:
os.getcwd()

'/home/tylan/Documents/lhl/projects/statistical-modelling-project/notebooks'

First I need a list of cities and the companies that operate in them. I send a request to the citybikes api requestion only the href (api extension for a particular company) and location information.

In [144]:
# req = requests.get('http://api.citybik.es/v2/networks/?fields=href,location')

# print(req)

The request returns a Response Object containing a status code and a JSON object. I use response.JSON() to convert the JSON response into a dictionary. All of the required data is at the dictionary key 'networks'.

In [145]:
# json_dict = req.json()
# data_list = json_dict['networks']
# pprint(data_list)

I use pd.json_normalize to convert the list of network dictionaries into a Pandas dataframe, networks_df.

In [146]:
# networks_df = pd.json_normalize(data_list)

In [147]:
# networks_df.head()

I want to use data for Tokyo, so I filter the networks_df for 'location.city' == 'Tokyo'. I find that there is only one company listed on the CityBikes API for Tokyo, 'Docomo Cycle Tokyo'.

In [148]:
# tokyo_networks = networks_df[networks_df['location.city'] == 'Tokyo']

In order to get the data on all of the Docomo Cycle Tokyo stations, I make a second request to the CityBikes API, this time specifying the href found above.

In [149]:
tokyo_req = requests.get('http://api.citybik.es/v2/networks/docomo-cycle-tokyo')

print(tokyo_req)

<Response [200]>


In [150]:
json_dict = tokyo_req.json()
pprint(json_dict)

{'network': {'company': ['DOCOMO BIKE SHARE, INC.'],
             'gbfs_href': 'https://api-public.odpt.org/api/v4/gbfs/docomo-cycle-tokyo/gbfs.json',
             'href': '/v2/networks/docomo-cycle-tokyo',
             'id': 'docomo-cycle-tokyo',
             'license': {'name': 'Creative Commons Attribution 4.0 '
                                 'International (CC BY 4.0)',
                         'url': 'https://creativecommons.org/licenses/by/4.0/'},
             'location': {'city': 'Tokyo',
                          'country': 'JP',
                          'latitude': 35.6827,
                          'longitude': 139.766},
             'name': 'Docomo Bike Share Tokyo',
             'source': 'https://ckan.odpt.org/en/dataset/c_bikeshare_gbfs-d-bikeshare',
             'stations': [{'empty_slots': 4,
                           'extra': {'last_updated': 1707443523,
                                     'renting': True,
                                     'returning': True,
  

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

The relevant data in the above dictionary is located at json_dict['network']['stations']. I can easily extract this info into a dataframe using json_normalize. I will also save the name of the company and location of its head office for future visualizations.

In [151]:
company_name = json_dict['network']['company']
home_office = json_dict['network']['location']

Put your parsed results into a DataFrame.

In [152]:
docomo_df = pd.json_normalize(json_dict['network']['stations'])
docomo_df

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.last_updated,extra.renting,extra.returning,extra.slots,extra.uid
0,4,0,c366861723241c80d8a3aca0bfdca57c,35.708596,139.758269,E5-26.本郷4丁目,2024-02-09T01:52:13.657000Z,1707443523,True,True,4,00010908
1,24,12,e5c85162a587db3bb1df3c15cb10a26f,35.692599,139.770109,A3-42.内神田3丁目,2024-02-09T01:52:13.668000Z,1707443523,True,True,36,00010909
2,3,2,9661f81050ae899a3fbafa1858b5876b,35.748736,139.574839,T2-76.東中前の森緑地,2024-02-09T01:52:13.669000Z,1707443523,True,True,5,00001809
3,2,15,2764578a43cb789efd7a884e1f11523e,35.693408,139.762809,A2-20.神田スクエア,2024-02-09T01:52:13.670000Z,1707443523,True,True,17,00010900
4,6,1,3684bb1938d533976c38d9e921232890,35.586993,139.736491,K1-79.Tマークシティホテル東京大森,2024-02-09T01:52:13.671000Z,1707443523,True,True,7,00010901
...,...,...,...,...,...,...,...,...,...,...,...,...
1319,0,3,9876ec50e4b95b976b96e9bad37011be,35.666349,139.741781,C3-36.第35興和ビル,2024-02-09T01:52:15.252000Z,1707443523,True,True,3,00002363
1320,30,76,f1cf36685d11745be7e127701ba86f45,35.651627,139.773103,B4-53.晴海五丁目ターミナル,2024-02-09T01:52:15.255000Z,1707443523,True,True,106,00002361
1321,9,0,a8cd61af1b8495529e65bbbc62dc3d6e,35.643105,139.753039,C5-44.ブランズタワー芝浦,2024-02-09T01:52:13.699000Z,1707443523,True,True,9,00000519
1322,11,2,457bfef68dbad019197e6676c18621c3,35.690602,139.766916,A3-47.ホテルヴィラフォンテーヌ東京大手町,2024-02-09T01:52:14.563000Z,1707443523,True,True,13,00002401


Some of this data isn't very useful for my analysis, so I will drop it.

In [153]:
docomo_df = docomo_df[['empty_slots', 'free_bikes', 'latitude', 'longitude', 'name', 'timestamp', 'extra.last_updated', 'extra.slots']]
docomo_df

,empty_slots,free_bikes,latitude,longitude,name,timestamp,extra.last_updated,extra.slots
0,4,0,35.708596,139.758269,E5-26.本郷4丁目,2024-02-09T01:52:13.657000Z,1707443523,4
1,24,12,35.692599,139.770109,A3-42.内神田3丁目,2024-02-09T01:52:13.668000Z,1707443523,36
2,3,2,35.748736,139.574839,T2-76.東中前の森緑地,2024-02-09T01:52:13.669000Z,1707443523,5
3,2,15,35.693408,139.762809,A2-20.神田スクエア,2024-02-09T01:52:13.670000Z,1707443523,17
4,6,1,35.586993,139.736491,K1-79.Tマークシティホテル東京大森,2024-02-09T01:52:13.671000Z,1707443523,7
...,...,...,...,...,...,...,...,...
1319,0,3,35.666349,139.741781,C3-36.第35興和ビル,2024-02-09T01:52:15.252000Z,1707443523,3
1320,30,76,35.651627,139.773103,B4-53.晴海五丁目ターミナル,2024-02-09T01:52:15.255000Z,1707443523,106
1321,9,0,35.643105,139.753039,C5-44.ブランズタワー芝浦,2024-02-09T01:52:13.699000Z,1707443523,9
1322,11,2,35.690602,139.766916,A3-47.ホテルヴィラフォンテーヌ東京大手町,2024-02-09T01:52:14.563000Z,1707443523,13


Lastly, I save the docomo_df data as a csv for future use.

In [154]:
docomo_df.to_csv('docomo_data' + str(docomo_df.loc[0, 'extra.last_updated']) + '.csv')